<a href="https://colab.research.google.com/github/eshnil2000/google-colab/blob/master/linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Install required modules
!pip install tensorflow numpy matplotlib pandas sklearn
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [18]:
# Parameters
learning_rate = 0.01
training_epochs = 3
display_step = 1

In [19]:
import pandas as pd
df = pd.read_csv('data/housepricedata.csv')
dataset = df.values
train_X = dataset[:,0:10]
train_Y = dataset[:,10]
batch_size=64
n_batches = int(train_X.shape[0]/batch_size)

In [20]:
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(train_X)
train_X=X_scale

In [21]:
depth=len(np.unique(train_Y))
indices=train_Y
onehot_Y=tf.one_hot(indices, depth)
with tf.Session() as sess:
    onehot_Y=sess.run(onehot_Y)
sess.close()

In [22]:
# tf Graph Input
X = tf.placeholder(tf.float32,shape=(batch_size,10), name="input")
Y = tf.placeholder(tf.float32,shape=(batch_size,2), name="label")

# Set model weights
# Create weights and bias
w = tf.Variable(tf.random_normal(shape=(10,1), stddev=0.01), name="weights")
b = tf.Variable(tf.zeros(shape=(1,2)), name='bias')

# calculate scores
logits = tf.matmul(X, w) + b

# Entropy cost function and loss
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y)
loss = tf.reduce_mean(entropy)

# Define optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

In [23]:
pred=logits
cost=loss

In [24]:
# Initialize the variables (i.e. assign their default value)
init  = tf.global_variables_initializer()
init2 = tf.initialize_all_variables()

In [25]:
##define the function that feeds the data to the model .
def train_input_fn(x_train,y_train,batch_size=64):
    ##Here we are using dataset API.
    '''
    take the data from tensor_slices i.e. an array of data-points in simple words.
    '''
    dataset =    tf.data.Dataset.from_tensor_slices((x_train,y_train)).batch(batch_size) 
    dataset_iterator = dataset.make_one_shot_iterator()   
    return dataset_iterator.get_next()

In [26]:
# Run optimization and test
loss_history = []
acc_history = []

# Start training
with tf.Session() as sess:
    sess.run(init)
    sess.run(init2)
    #tf.train.start_queue_runners(sess)

    #sess.run(train_Y)
    # Fit all training data
    for epoch in range(training_epochs):
        #for (x, y) in zip(train_X, train_Y):
        #for (x, y) in zip(train_X, onehot_Y): 
        for i in range(n_batches):
            slicex, slicey=train_input_fn(train_X,onehot_Y)
            sess.run(optimizer, feed_dict={X: slicex.eval(), Y: slicey.eval()})
            #sess.run(optimizer, feed_dict={X: x.reshape(1,10), Y: y.reshape(1,2)})
            
            _, loss_value = sess.run([optimizer, loss], feed_dict={X: slicex.eval(), Y: slicey.eval()})
        loss_history.append(loss_value)
    

In [27]:
loss_history

[0.6930591, 0.69299155, 0.6929419]

In [28]:
# Prediction
with tf.Session() as sess:
    sess.run(init)
    sess.run(init2)
    total_correct_preds = 0

    for i in range(n_batches):
        slicex, slicey=train_input_fn(train_X,onehot_Y)
        logits_batch = sess.run(logits, feed_dict={X: slicex.eval(), Y: slicey.eval()})
        preds = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(slicey.eval(), 1))
        accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
        total_correct_preds += sess.run(accuracy)
        
    print("Test accuracy is {0}".format(total_correct_preds/train_X.shape[0]))

Test accuracy is 0.3767123287671233
